In [1]:
import pandas as pd
import joblib
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
# from imblearn.under_sampling import RandomUnderSampler
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline

raw = pd.read_csv('post_PCA.csv')

In [5]:
%%time
from collections import Counter
# import random
from imblearn.under_sampling import RandomUnderSampler

testset = raw
scaler = joblib.load('Models/scaler.joblib')
# rand = random.randint(0, 99989)
# testset = raw[raw.index.isin(analysis[analysis['cluster'] == 0].index)]
# testset = raw[raw['REPUR_FLG']]

# testset = raw.loc[rand:rand+19]
# print(testset.head())
# print('Ground Truth:' , Counter(testset['Y1_repurchase']))
# print('Ground Truth:' , Counter(testset['REPUR_FLG']))
testset = testset.drop(['Y1_repurchase', 'REPUR_FLG'], axis=1)
testset_scaled = scaler.transform(testset)
y = raw['REPUR_FLG']

# under sampling
rus = RandomUnderSampler(random_state=42)
X, y = rus.fit_resample(testset_scaled, y)

for model in ['Logistic_Regression_0.97', 'Decision_Tree_0.86', 'XGBoost_0.92', 'SVM_0.97', "SVD+Logistic_Regression_0.98"]:
    logi = joblib.load(f'Models/{model}.joblib')

    predictions = logi.predict(X)
    # predictions = logi.predict(testset_scaled)
#     pred_probabilities = logi.predict_proba(testset_scaled)
    
    print("Model: ", model)
    print(classification_report(y, predictions))
    # print("Predictions:", Counter(predictions))
    # print("Prediction Probabilities:", pred_probabilities)

Model:  Logistic_Regression_0.97
              precision    recall  f1-score   support

       False       0.50      0.94      0.65      4999
        True       0.48      0.06      0.10      4999

    accuracy                           0.50      9998
   macro avg       0.49      0.50      0.38      9998
weighted avg       0.49      0.50      0.38      9998

Model:  Decision_Tree_0.86
              precision    recall  f1-score   support

       False       0.51      0.83      0.63      4999
        True       0.53      0.19      0.28      4999

    accuracy                           0.51      9998
   macro avg       0.52      0.51      0.45      9998
weighted avg       0.52      0.51      0.45      9998

Model:  XGBoost_0.92
              precision    recall  f1-score   support

       False       0.50      0.88      0.64      4999
        True       0.51      0.12      0.19      4999

    accuracy                           0.50      9998
   macro avg       0.50      0.50      0.42    

In [9]:
analysis = pd.read_csv('output/Repurchase_Clustering.csv',index_col=0)
analysis.groupby('cluster').mean()

,AGE,GENDER,CLIENT_INCOME,total_aum,ternure_m,recency_m,topcard,UNDERTAKE,LABEL_NUM,SIN1_POL_CNT,...,contact_PC8,contact_PC9,contact_PC10,contact_PC11,contact_PC12,contact_PC13,contact_PC14,contact_PC15,contact_PC16,Y1_repurchase
cluster,,,,,,,,,,,,,,,,,,,,,
-1,54.142857,0.285714,676944.857143,951925.142857,84.571429,87.285714,0.000000,0.285714,3.000000,0.285714,...,13.060186,3.668576,-0.691630,-2.275668,0.591107,-0.059721,0.387014,0.302108,0.156587,1.0
0,36.654102,0.534368,614664.904656,572083.496674,137.390244,140.261641,0.013304,0.299335,0.687361,0.048780,...,-0.027823,-0.123674,0.000551,-0.035285,-0.019677,-0.013372,-0.009801,-0.044850,-0.017671,1.0


In [7]:
def feature_filter(model, df):
    # 如果模型具有係數屬性，例如邏輯迴歸
    if hasattr(model, 'coef_'):
        importance = model.coef_[0]
        feature_importance = pd.DataFrame({'Feature': df.drop(['Y1_repurchase', 'REPUR_FLG'], axis=1).columns, 'Importance': importance})
        feature_importance = feature_importance.sort_values(by='Importance', ascending=False)
        print("Feature Importance:")
        print(feature_importance[:3])
    
    # 如果模型具有特徵重要性屬性，例如隨機森林和XGBoost
    elif hasattr(model, 'feature_importances_'):
        importance = model.feature_importances_
        feature_importance = pd.DataFrame({'Feature': df.drop(['Y1_repurchase', 'REPUR_FLG'], axis=1).columns, 'Importance': importance})
        feature_importance = feature_importance.sort_values(by='Importance', ascending=False)
        print("Feature Importance:")
        print(feature_importance[:3])

In [13]:
%%time
random_state = 313
df = raw
# X = df.drop(['Y1_repurchase', 'recency_m'], axis=1)
X = df.drop(['Y1_repurchase', 'REPUR_FLG'], axis=1)
y = df['Y1_repurchase']
# y = df['REPUR_FLG']
# y = df.index.isin(analysis[analysis['cluster'] == 0].index)

# 標準化特徵
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
# joblib.dump(scaler, 'Models/scaler.joblib')
# joblib.dump(scaler, 'Models/RF_scaler.joblib')

rus = RandomUnderSampler(random_state=random_state)
X_scaled, y = rus.fit_resample(X_scaled, y)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=random_state)

models = {
    'Logistic Regression': LogisticRegression(C=1000, max_iter=1000),
    'Random Forest': RandomForestClassifier(max_depth=80, n_estimators=300),
    # 'Random Forest': RandomForestClassifier(max_depth=80, n_estimators=200),
    'KNN': KNeighborsClassifier(n_neighbors=5),
    # 'KNN': KNeighborsClassifier(n_neighbors=9),
    'Decision Tree': DecisionTreeClassifier(max_depth=8, min_samples_split=3),
    # 'Decision Tree': DecisionTreeClassifier(max_depth=8, min_samples_split=2),
    'XGBoost': XGBClassifier(learning_rate=0.15, max_depth=4, n_estimators=200),
    # 'XGBoost': XGBClassifier(learning_rate=0.13, max_depth=3, n_estimators=150),
    'SVM': SVC(C=500, gamma=0.1, kernel='linear', probability=True), # probability=True
    # 'SVM': SVC(C=800, gamma=0.1, kernel='linear', probability=True),
    'SVD+Logistic Regression': make_pipeline(TruncatedSVD(n_components=50), LogisticRegression(C=1000, max_iter=1000))
}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
#     print(name, accuracy)
    print(f"Model: {name}")
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    print(f"Accuracy: {accuracy:.2f}")
    
    feature_filter(model, df)
    
    # joblib.dump(model, f"Models/RF_{name.replace(' ', '_')}_{accuracy:.2f}.joblib")
    print()

Model: Logistic Regression
[[88  5]
 [ 1 90]]
              precision    recall  f1-score   support

       False       0.99      0.95      0.97        93
        True       0.95      0.99      0.97        91

    accuracy                           0.97       184
   macro avg       0.97      0.97      0.97       184
weighted avg       0.97      0.97      0.97       184

Accuracy: 0.97
Feature Importance:
               Feature  Importance
5            recency_m  117.749987
50  policy_ex_RATE_PC9   13.655152
82        contact_PC14   12.217374

Model: Random Forest
[[77 16]
 [ 4 87]]
              precision    recall  f1-score   support

       False       0.95      0.83      0.89        93
        True       0.84      0.96      0.90        91

    accuracy                           0.89       184
   macro avg       0.90      0.89      0.89       184
weighted avg       0.90      0.89      0.89       184

Accuracy: 0.89
Feature Importance:
      Feature  Importance
28  label_PC4    0.1032

In [60]:
wrong_indices = [i for i, (pred, true) in enumerate(zip(y_pred, y_test)) if pred != true]
print(f"Wrongly predicted indices: {wrong_indices}")

Wrongly predicted indices: [7, 16, 21, 32, 38, 41, 43, 57, 60, 62, 76, 96, 98, 103, 113, 114, 115, 118, 119, 138, 140, 157, 158, 178]


In [11]:
%%time
from sklearn.model_selection import GridSearchCV
param_grid = {
    # 'Logistic Regression': {'C': [500, 1000]},
    # 'Random Forest': {'n_estimators': [150, 200, 300, 450], 'max_depth': [40, 50, 60, 80]},
    # 'KNN': {'n_neighbors': [3, 5, 7, 9]},
    # 'Decision Tree': {'max_depth': [8, 10, 12], 'min_samples_split': [2, 3, 4]},
    # 'XGBoost': {'learning_rate': [0.13, 0.15, 0.2, 0.5], 
    #             'n_estimators': [150, 200, 250], 'max_depth': [3, 4, 5]},
    # 'SVM': {'C': [350, 500, 800],'kernel': ['linear', 'rbf', 'poly'],
    #           'gamma': [0.1, 0.15, 0.08]},
    'SVD+Logistic Regression': {'logisticregression__C': [200, 300, 500, 1000]}
}

# 使用網格搜索進行參數優化
for name, model in models.items():
    if name in param_grid:
        grid_search = GridSearchCV(model, param_grid[name], cv=5, scoring='accuracy')
        grid_search.fit(X_train, y_train)
        best_params = grid_search.best_params_
        best_score = grid_search.best_score_
        print(f"Best parameters for {name}: {best_params}")
        print(f"Best cross-validation accuracy: {best_score:.2f}")

Best parameters for SVD+Logistic Regression: {'logisticregression__C': 1000}
Best cross-validation accuracy: 0.97
CPU times: total: 93.8 ms
Wall time: 852 ms
